In [ ]:
# CELDA 1: GENERACIÓN DEL DATASET SINTÉTICO
# -----------------------------------------

## IMPORTACIÓN DE LIBRERÍAS PARA ANÁLISIS
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


## PARÁMETROS DE SIMULACIÓN
TOTAL_CLIENTES = 5000
FECHA_INICIO_SIM = "2022-01-01"
FECHA_FIN_SIM = "2025-11-10"  # Un día antes de tu fecha de análisis
COSTO_PROMEDIO_SERVICIO = 250
DESVIACION_COSTO = 150
output_filename = "transacciones_servicio_honda.csv" # Nombre del archivo que se creará

print(f"Iniciando simulación con {TOTAL_CLIENTES} clientes...")


## DEFINICIÓN DE PERFILES DE SEGMENTO
segment_profiles = {
    "VIP": {"ratio": 0.10, "txns_rango": (10, 20), "recencia_dias": (1, 90), "gasto_mult": (1.5, 3.0)},
    "En Riesgo": {"ratio": 0.20, "txns_rango": (8, 15), "recencia_dias": (365, 730), "gasto_mult": (1.2, 2.5)},
    "Nuevos": {"ratio": 0.15, "txns_rango": (1, 2), "recencia_dias": (1, 180), "gasto_mult": (0.8, 1.2)},
    "Leales Promedio": {"ratio": 0.30, "txns_rango": (5, 10), "recencia_dias": (30, 180), "gasto_mult": (1.0, 1.5)},
    "Bajo Valor (Perdidos)": {"ratio": 0.25, "txns_rango": (1, 3), "recencia_dias": (730, 1080), "gasto_mult": (0.7, 1.0)}
}


## GENERACIÓN DE CLIENTES
start_date = datetime.strptime(FECHA_INICIO_SIM, "%Y-%m-%d")
snapshot_date = datetime.strptime(FECHA_FIN_SIM, "%Y-%m-%d")

customer_list = []
cliente_id_counter = 1
for segment_name, profile in segment_profiles.items():
    num_clientes_en_segmento = int(TOTAL_CLIENTES * profile["ratio"])
    for _ in range(num_clientes_en_segmento):
        customer_list.append({"CustomerID": f"CUST_{cliente_id_counter:05d}", "SegmentoAsignado": segment_name})
        cliente_id_counter += 1

while len(customer_list) < TOTAL_CLIENTES:
    customer_list.append({"CustomerID": f"CUST_{cliente_id_counter:05d}", "SegmentoAsignado": "Leales Promedio"})
    cliente_id_counter += 1

df_clientes = pd.DataFrame(customer_list).sample(frac=1).reset_index(drop=True)


## GENERACIÓN DE TRANSACCIONES
transactions_list = []
for _, cliente in df_clientes.iterrows():
    customer_id = cliente["CustomerID"]
    segment_name = cliente["SegmentoAsignado"]
    profile = segment_profiles[segment_name]
    num_transacciones = random.randint(*profile["txns_rango"])
    gasto_mult_cliente = random.uniform(*profile["gasto_mult"])
    dias_recencia = random.randint(*profile["recencia_dias"])
    fecha_ultima_txn = snapshot_date - timedelta(days=dias_recencia)

    monto = max(50, np.random.normal(COSTO_PROMEDIO_SERVICIO * gasto_mult_cliente, DESVIACION_COSTO))
    transactions_list.append({"CustomerID": customer_id, "TransactionDate": fecha_ultima_txn, "Amount": round(monto, 2)})

    if num_transacciones > 1:
        dias_entre_inicio_y_ultima_txn = (fecha_ultima_txn - start_date).days
        if dias_entre_inicio_y_ultima_txn > 30:
            for _ in range(num_transacciones - 1):
                dias_offset = random.randint(1, dias_entre_inicio_y_ultima_txn)
                fecha_txn = start_date + timedelta(days=dias_offset)
                monto = max(50, np.random.normal(COSTO_PROMEDIO_SERVICIO * gasto_mult_cliente, DESVIACION_COSTO))
                transactions_list.append({"CustomerID": customer_id, "TransactionDate": fecha_txn, "Amount": round(monto, 2)})


## GUARDAR EL DATASET
df_transacciones = pd.DataFrame(transactions_list).sample(frac=1).reset_index(drop=True)
df_transacciones.to_csv(output_filename, index=False)

print(f"\n¡Éxito! Se generaron {len(df_transacciones)} transacciones.")
print(f"Dataset guardado como: {output_filename}")

Iniciando simulación con 5000 clientes...

¡Éxito! Se generaron 33711 transacciones.
Dataset guardado como: transacciones_servicio_honda.csv


In [2]:
# CELDA 2: CÁLCULO DE MÉTRICAS RFM
# --------------------------------

## IMPORTACIÓN DE LIBRERÍAS PARA VISUALIZACIÓN
import matplotlib.pyplot as plt
import seaborn as sns


## CARGA Y PREPARACIÓN
df = pd.read_csv('transacciones_servicio_honda.csv')
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
snapshot_date = pd.to_datetime('2025-11-11')

print("Archivo CSV cargado exitosamente.")
print(df.head())

Archivo CSV cargado exitosamente.
   CustomerID TransactionDate  Amount
0  CUST_00251      2025-11-04  618.01
1  CUST_01376      2022-09-30  495.00
2  CUST_00464      2024-03-04  434.23
3  CUST_00195      2024-12-03  333.75
4  CUST_04043      2023-03-16  176.49


In [3]:
# CELDA 3: CÁLCULO DE RFM
# -----------------------

rfm_df = df.groupby('CustomerID').agg(
    Recency_days=('TransactionDate', lambda x: (snapshot_date - x.max()).days),
    Frequency=('TransactionDate', 'count'),
    Monetary=('Amount', 'sum')
).reset_index()

print("\nTabla RFM calculada:")
print(rfm_df.head())


Tabla RFM calculada:
   CustomerID  Recency_days  Frequency  Monetary
0  CUST_00001            37         10   4699.56
1  CUST_00002             5         15   7203.94
2  CUST_00003            13         16  11122.66
3  CUST_00004            34         10   4435.21
4  CUST_00005            44         10   3400.44


In [4]:
# CELDA 4: PUNTUACIÓN Y SEGMENTACIÓN
# ----------------------------------

## PUNTUACIÓN (SCORING)
rfm_df['R_Score'] = pd.qcut(rfm_df['Recency_days'], 4, labels=[4, 3, 2, 1])
rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm_df['RFM_Score'] = rfm_df['R_Score'].astype(str) + rfm_df['F_Score'].astype(str) + rfm_df['M_Score'].astype(str)


## SEGMENTACIÓN ESTRATÉGICA
segment_map = {
    r'[3-4][3-4][3-4]': 'Clientes VIP',
    r'[3-4]1[1-2]': 'Nuevos Clientes',
    r'[1-2][3-4][3-4]': 'En Riesgo de Fuga (Valiosos)',
    r'[3-4][2-3][2-3]': 'Clientes Leales (Promedio)',
    r'1[1-2][1-2]': 'Clientes Perdidos (Bajo Valor)',
    r'[1-2][1-2][3-4]': 'Clientes "Ahorradores" Perdidos'
}

def assign_segment(rfm_score):
    for pattern, segment in segment_map.items():
        if pd.Series([rfm_score]).str.match(pattern).any():
            return segment
    return 'Otros (En Desarrollo)'

rfm_df['Segmento'] = rfm_df['RFM_Score'].apply(assign_segment)


## ALMACENAMIENTO DEL RESULTADO FINAL
rfm_df.to_csv('rfm_segmentacion_clientes.csv', index=False)

print("\n¡Segmentación completada!")
print(rfm_df.head())


¡Segmentación completada!
   CustomerID  Recency_days  Frequency  Monetary R_Score F_Score M_Score  \
0  CUST_00001            37         10   4699.56       4       3       4   
1  CUST_00002             5         15   7203.94       4       4       4   
2  CUST_00003            13         16  11122.66       4       4       4   
3  CUST_00004            34         10   4435.21       4       3       4   
4  CUST_00005            44         10   3400.44       4       3       3   

  RFM_Score      Segmento  
0       434  Clientes VIP  
1       444  Clientes VIP  
2       444  Clientes VIP  
3       434  Clientes VIP  
4       433  Clientes VIP  


In [18]:
# CELDA 5: ANÁLISIS DE RESULTADOS
# -------------------------------

print("\n--- ANÁLISIS DE RESULTADOS ---")
print("\n-----------------------------------------------------------------------------")

## DETERMINAR NÚMERO DE CLIENTES POR SEGMENTO
segment_counts = rfm_df['Segmento'].value_counts().reset_index()
segment_counts.columns = ['Segmento', 'Conteo_Clientes']

print("\n1) Conteo de clientes por segmento:")
print("")
print(segment_counts)
print("")
print("\n-----------------------------------------------------------------------------")


## CONSULTA DE STATS PROMEDIAS EN RECENCIA, FRECUENCIA Y VALOR MONETARIO POR SEGMENTO
segment_means = rfm_df.groupby('Segmento')[['Recency_days', 'Frequency', 'Monetary']].mean().reset_index()
segment_means = segment_means.round(1)

print("\n2) Promedios por segmento:")
print("")
print(segment_means)
print("")


--- ANÁLISIS DE RESULTADOS ---

-----------------------------------------------------------------------------

1) Conteo de clientes por segmento:

                          Segmento  Conteo_Clientes
0                     Clientes VIP             1267
1   Clientes Perdidos (Bajo Valor)             1247
2     En Riesgo de Fuga (Valiosos)             1097
3                  Nuevos Clientes              669
4       Clientes Leales (Promedio)              568
5            Otros (En Desarrollo)              139
6  Clientes "Ahorradores" Perdidos               13


-----------------------------------------------------------------------------

2) Promedios por segmento:

                          Segmento  Recency_days  Frequency  Monetary
0  Clientes "Ahorradores" Perdidos         169.2        5.8    2261.2
1       Clientes Leales (Promedio)          95.9        5.9    1729.3
2   Clientes Perdidos (Bajo Valor)         909.1        2.0     452.6
3                     Clientes VIP          77